#### Generating List of sentences

In [1]:
import xml.etree.ElementTree as ET
print('Processing text dataset')

tree = ET.parse("/home/jeet/Academics/CS671/Project/Restaurants_Train.xml")
corpus = tree.getroot()
sentences = [] # List of sentences.
raw_data = corpus.findall('.//sentence')
for sent in raw_data:
    sentences.append(sent.find('text').text)

print ('Generated list of sentences..')

Processing text dataset
Generated list of sentences..


#### Indexing Aspects

In [2]:
import nltk
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

train_out= np.zeros(shape=(3044,69))
i=0
for output in raw_data:
    s = text_to_word_sequence(output.find('text').text, lower=False)
    indices = np.zeros(69)
    
    aspectTerms = output.find('aspectTerms')
    if (aspectTerms):
        aspectTerm = aspectTerms.findall('aspectTerm')
        if (aspectTerm):
            for aspect_term in aspectTerm:
                try:
                    indices[s.index(aspect_term.attrib['term'])] = 1
#                     print (indices)
                except:
                    continue
    train_out[i] = indices
    i=i+1

print ("Shape of output tensor:", train_out.shape)

Using Theano backend.
Using gpu device 0: GeForce GT 730M (CNMeM is disabled, cuDNN 5103)


('Shape of output tensor:', (3044, 69))


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:12: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.


#### Defining input data for CRF Training

In [17]:
train_input = []
i = 0
for sent in sentences:
    tuple_sent = []
    word_token = text_to_word_sequence(sent, lower=False)
    j = 0
    for word in word_token:
        input_tuple = ()
        input_tuple = input_tuple + (word.decode('UTF-8'),)
        if train_out[i][j] == 0:
            input_tuple = input_tuple + ('NA',)
        else:
            input_tuple = input_tuple + ('A',)
        tuple_sent.append(input_tuple)
        j=j+1
    train_input.append(tuple_sent)
    i=i+1

In [18]:
print train_input[:5]

[[(u'But', 'NA'), (u'the', 'NA'), (u'staff', 'A'), (u'was', 'NA'), (u'so', 'NA'), (u'horrible', 'NA'), (u'to', 'NA'), (u'us', 'NA')], [(u'To', 'NA'), (u'be', 'NA'), (u'completely', 'NA'), (u'fair', 'NA'), (u'the', 'NA'), (u'only', 'NA'), (u'redeeming', 'NA'), (u'factor', 'NA'), (u'was', 'NA'), (u'the', 'NA'), (u'food', 'A'), (u'which', 'NA'), (u'was', 'NA'), (u'above', 'NA'), (u'average', 'NA'), (u'but', 'NA'), (u"couldn't", 'NA'), (u'make', 'NA'), (u'up', 'NA'), (u'for', 'NA'), (u'all', 'NA'), (u'the', 'NA'), (u'other', 'NA'), (u'deficiencies', 'NA'), (u'of', 'NA'), (u'Teodora', 'NA')], [(u'The', 'NA'), (u'food', 'A'), (u'is', 'NA'), (u'uniformly', 'NA'), (u'exceptional', 'NA'), (u'with', 'NA'), (u'a', 'NA'), (u'very', 'NA'), (u'capable', 'NA'), (u'kitchen', 'A'), (u'which', 'NA'), (u'will', 'NA'), (u'proudly', 'NA'), (u'whip', 'NA'), (u'up', 'NA'), (u'whatever', 'NA'), (u'you', 'NA'), (u'feel', 'NA'), (u'like', 'NA'), (u'eating', 'NA'), (u'whether', 'NA'), (u"it's", 'NA'), (u'on', 'N

In [11]:
train_data = train_input[:2739]
test_data = train_input[2739:]
true_test = sentences[2739:]

In [19]:
from nltk.tag import CRFTagger
ct = CRFTagger()

ct.train(train_data,'model_Crf_tagger')

In [25]:
ct.tag_sents(true_test)

TypeError: must be unicode, not str